In [1]:
import spacy
import re
import nltk
import numpy as np
import pandas as pd
from pprint import pprint
import pickle
import gensim
import logging
import warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamodel import LdaModel
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
import pyLDAvis
from gensim.models.phrases import Phraser


C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
import numpy as np
print(np.__version__)


1.19.2


In [3]:
# from spacy.cli import download
# download("en_core_web_sm")

In [4]:
%tb

No traceback available to show.


In [52]:
df = pd.read_parquet("../data/nation.parquet")

In [53]:
stopwords = nltk.corpus.stopwords.words('english')

In [54]:
def removeSpecialChars(sentence):
    return sentence.replace("\n"," ")

In [55]:
def remove_emails_nl_quotes(sentence):
    if (sentence is np.nan):
        return
    sentence = re.sub('\S*@\S*\s?', '', sentence)
    sentence = re.sub('\s+', ' ', sentence)
    sentence = re.sub("\'", "", sentence)
    sentence = gensim.utils.simple_preprocess(str(sentence), deacc=True)
    return sentence



<>:4: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \S
<>:5: DeprecationWarning: invalid escape sequence \s
C:\Users\tommy\AppData\Local\Temp\ipykernel_8388\3160853986.py:4: DeprecationWarning: invalid escape sequence \S
  sentence = re.sub('\S*@\S*\s?', '', sentence)
C:\Users\tommy\AppData\Local\Temp\ipykernel_8388\3160853986.py:5: DeprecationWarning: invalid escape sequence \s
  sentence = re.sub('\s+', ' ', sentence)


In [56]:
def remove_urls(text):
    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+|www\.[a-zA-Z0-9./]+'
    clean_text = re.sub(url_pattern, '', text)
    return clean_text

In [57]:
def word_tokenization(sentence):
    return [token for token in sentence if token not in stopwords]


In [145]:
def preprocess(text):
    text = removeSpecialChars(text)
    text = remove_emails_nl_quotes(text)
    text = word_tokenization(text)

    nlp = spacy.load("en_core_web_sm")
    allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
    
    doc = nlp(" ".join(text))
    print([ i for i in doc])
    texts_out = [token.lemma_ for token in doc]
    return texts_out

In [146]:
preprocess("hello hello hello")

[hello, hello, hello]


['hello', 'hello', 'hello']

In [79]:
def preprocessing(df, paragraph_col):
    df["no_special_chars"] = df[paragraph_col].apply(removeSpecialChars)
    df['Preprocessed'] = df["no_special_chars"].apply(remove_emails_nl_quotes)
    df['No_Stopwords'] = df['Preprocessed'].apply(word_tokenization)
    data_words = df['No_Stopwords'].values
    bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
    bigram_model = gensim.models.phrases.Phraser(bigram)
    trigram_model = gensim.models.phrases.Phraser(trigram)
    
    texts = data_words
    texts = [bigram_model[doc] for doc in texts]
    texts = [trigram_model[bigram_model[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load("en_core_web_sm")
    allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append(
            [token.lemma_ for token in doc if token.pos_ in allowed_postags])
        texts_out = [[word for word in simple_preprocess(
            str(doc)) if word not in stopwords] for doc in texts_out]

    data_ready = texts_out
    return data_ready


In [80]:
data = preprocessing(df, "paragraphs")

0                             No purchases Pleaselog in
1     Daniel Mururu is accused of radicalisation, de...
2     President William Ruto shakes hands with Chine...
3     President William Ruto shakes hands with Chine...
4     President William Ruto shakes hands with Chine...
5     President William Ruto shakes hands with Chine...
6     Some parents have managed to salvage some form...
7     Certain HPV Strains cause cancers of the throa...
8     In this episode, we dive into the world of in-...
9     Inspector General of Police Japhet Koome on Tu...
10    Through exploring the life of Jaramogi Oginga ...
11    On this episode Leila Mohammed, Mary Wambui an...
12    In Nakukulalas, Turkana the center of Kenya's ...
13    In this episode of "Money Moves," your host Br...
14    George Gachui, the co-founder of Mookh Africa,...
15    On this episode of NFM Drive Chat, Dr. Ajwa Ha...
16    In this episode, ELIAS MAKORI speaks with Jona...
17    This week we travel all the way to East As

In [64]:
data

[['purchase', 'pleaselog'],
 ['accuse',
  'radicalisation',
  'defile',
  'underage',
  'girl',
  'rape',
  'woman',
  'accuse',
  'radicalisation',
  'defile',
  'underage',
  'girl',
  'rape',
  'argue',
  'high',
  'court',
  'order',
  'violate',
  'natural',
  'justice',
  'exclude',
  'legislative',
  'body',
  'proceeding',
  'pastor',
  'bizarre',
  'ritual',
  'blood',
  'sprinkling',
  'force',
  'sexual',
  'act',
  'friend',
  'plan',
  'occasion',
  'mark',
  'new',
  'chapter',
  'live',
  'kenha',
  'official',
  'outline',
  'option',
  'end',
  'slaughter',
  'part',
  'convoy',
  'carry',
  'group',
  'friend',
  'parental',
  'blessing',
  'ceremony',
  'victim',
  'succumb',
  'deep',
  'machete',
  'cut',
  'angry',
  'villager',
  'beat',
  'assailant',
  'death',
  'resident',
  'accuse',
  'pastor',
  'engage',
  'activity',
  'border',
  'cultism',
  'neighbour',
  'try',
  'put',
  'fire',
  'strong',
  'wind',
  'make',
  'inferno',
  'spread',
  'quickly',
 

In [81]:
id2word = corpora.Dictionary(data)

In [82]:
id2word

In [83]:
corpus = [id2word.doc2bow(text) for text in data ]

In [84]:
corpus

[[(0, 1), (1, 1)],
 [(2, 3),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 2),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 2),
  (42, 2),
  (43, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 2),
  (76, 2),
  (77, 2),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 2),
  (82, 1),
  (83, 1),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 1),

In [104]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=10, random_state=100, chunksize=100, passes=20, alpha='auto', per_word_topics=True)

In [105]:
pprint(lda_model.print_topics())

[(0,
  '0.024*"police" + 0.015*"business" + 0.015*"man" + 0.015*"life" + '
  '0.015*"share" + 0.015*"build" + 0.013*"sight_public" + 0.013*"come" + '
  '0.013*"acting_swiftly" + 0.013*"visit"'),
 (1,
  '0.016*"food" + 0.016*"know" + 0.016*"impact" + 0.016*"well" + 0.016*"come" '
  '+ 0.016*"police" + 0.008*"economy" + 0.008*"back" + 0.008*"way" + '
  '0.008*"future"'),
 (2,
  '0.055*"police" + 0.025*"talk" + 0.021*"olympian_blames" + '
  '0.021*"save_daughter" + 0.021*"eye_resurfacing" + 0.021*"matter" + '
  '0.021*"acting_swiftly" + 0.021*"visit" + 0.021*"family_murdere" + '
  '0.021*"dp_gachagua"'),
 (3,
  '0.001*"episode" + 0.001*"talk" + 0.001*"year" + 0.001*"take" + '
  '0.001*"family" + 0.001*"explore" + 0.001*"chinese" + 0.001*"money" + '
  '0.001*"country" + 0.001*"man"'),
 (4,
  '0.019*"episode" + 0.010*"explore" + 0.009*"take" + 0.009*"family" + '
  '0.009*"story" + 0.009*"man" + 0.009*"talk" + 0.005*"court" + 0.005*"accuse" '
  '+ 0.005*"new"'),
 (5,
  '0.001*"year" + 0.001*

In [106]:
dictionary = corpora.Dictionary(data)

In [107]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [152]:
new_doc_tokens = preprocess("Someone once told me that you had a boyfriend.")
new_doc_bow = dictionary.doc2bow(new_doc_tokens)
new_doc_topics = lda_model.get_document_topics(new_doc_bow)

[someone, told, boyfriend]


In [153]:
new_doc_topics

[(0, 0.02589841),
 (1, 0.017609067),
 (2, 0.03572549),
 (3, 0.0142023135),
 (4, 0.7898515),
 (5, 0.014031267),
 (6, 0.031367008),
 (7, 0.017789118),
 (8, 0.021289755),
 (9, 0.032236136)]